In [138]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from typing import Optional, List, Callable, Any, Union, Dict
from itertools import product
from statistics import mean
from pathlib import Path
import gzip
import os
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

### Load and preprocess data

In [125]:
data = pd.read_csv('data/output/combined.csv')
data.head()

,battleneturl,Base,SingleMineral,hotkey00,hotkey01,hotkey02,hotkey10,hotkey11,hotkey12,hotkey20,...,hotkey71,hotkey72,hotkey80,hotkey81,hotkey82,hotkey90,hotkey91,hotkey92,s,0
0,53.0,0.029570,0.002240,0.002240,0.0,0.018369,0.028226,0.000000,0.156810,0.016577,...,0.0,0.0,0.000000,0.0,0.000000,0.001792,0.0,0.000000,0.301971,0.082717
1,29.0,0.005778,0.000000,0.000963,0.0,0.039961,0.031295,0.001444,0.215696,0.007222,...,0.0,0.0,0.000481,0.0,0.007703,0.000481,0.0,0.008666,0.259027,0.061559
2,53.0,0.034866,0.002226,0.002226,0.0,0.013353,0.012611,0.000000,0.096439,0.006677,...,0.0,0.0,0.000000,0.0,0.000000,0.001484,0.0,0.001484,0.318991,0.049539
3,29.0,0.015130,0.000000,0.001441,0.0,0.028098,0.023055,0.000720,0.186599,0.010086,...,0.0,0.0,0.000000,0.0,0.000000,0.000720,0.0,0.010086,0.286023,0.059794
4,53.0,0.003231,0.000000,0.001616,0.0,0.000000,0.016155,0.000000,0.084006,0.009693,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.336026,0.051345


In [126]:
#filter dataframe by non important features

#len(importance_df)
#filtered_df = pd.concat([data[importance_df['Feature'].head(25)], data['battleneturl']], axis=1)
#data = filtered_df

In [127]:
# Split the data into training and testing sets
X = data.drop(['battleneturl'], axis=1)
#X = data.drop(['battleneturl', 'played_race_Protoss', 'played_race_Terran', 'played_race_Zerg'], axis=1)
y = data['battleneturl']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [128]:
X_train

,Base,SingleMineral,hotkey00,hotkey01,hotkey02,hotkey10,hotkey11,hotkey12,hotkey20,hotkey21,...,hotkey71,hotkey72,hotkey80,hotkey81,hotkey82,hotkey90,hotkey91,hotkey92,s,0
781,0.004968,0.000000,0.000000,0.000000,0.000000,0.013485,0.000710,0.090135,0.002129,0.000000,...,0.000000,0.006388,0.000710,0.000000,0.014904,0.000710,0.000000,0.000710,0.225692,0.037823
2629,0.001032,0.020647,0.000344,0.000000,0.000000,0.020303,0.000000,0.150723,0.002753,0.000000,...,0.000000,0.000000,0.000344,0.000000,0.002065,0.000344,0.000000,0.000000,0.380936,0.057604
227,0.000494,0.000494,0.000000,0.000000,0.000000,0.012352,0.001482,0.066206,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.285573,0.024931
365,0.004637,0.000000,0.000000,0.000000,0.000000,0.010268,0.016893,0.184167,0.001325,0.000662,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.406095,0.023609
2404,0.029160,0.000389,0.000000,0.000000,0.000000,0.029938,0.000000,0.194012,0.003110,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.489502,0.076799
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1638,0.000000,0.005030,0.001341,0.000000,0.019785,0.001006,0.000000,0.144534,0.031187,0.000335,...,0.000000,0.007042,0.000671,0.000000,0.007378,0.000671,0.000000,0.006707,0.316566,0.081068
1095,0.020039,0.000786,0.000000,0.000000,0.000000,0.007073,0.007073,0.097446,0.006287,0.001965,...,0.000000,0.011788,0.000393,0.000000,0.000000,0.000000,0.000000,0.000000,0.447544,0.040650
1130,0.002118,0.000000,0.000000,0.001059,0.029645,0.010058,0.008470,0.194283,0.017470,0.000000,...,0.000529,0.012176,0.000000,0.000529,0.000000,0.000000,0.000529,0.000000,0.197988,0.040371
1294,0.000000,0.041782,0.000307,0.000000,0.000000,0.010138,0.000000,0.150845,0.006452,0.000000,...,0.000000,0.000000,0.000307,0.000000,0.002458,0.000307,0.000000,0.000000,0.282949,0.033212


In [129]:
# Ensure all column names in X_train are strings
X_train.columns = X_train.columns.astype(str)

# Repeat the same for X_test if applicable
X_test.columns = X_test.columns.astype(str)

### Train the model

In [149]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
import numpy as np

# Define a small set of hyperparameters to tune
param_dist = {
    'n_estimators': [100, 300, 500],  # Small range for faster tuning
    'max_depth': [None, 10, 20],  # Limit depth tuning
    'min_samples_split': [2, 5, 10],  
    'min_samples_leaf': [1, 2, 4],  
}

# Create the Random Forest model
rf = RandomForestClassifier(random_state=42)

# Perform Randomized Search (only 10 trials)
random_search = RandomizedSearchCV(
    rf, param_distributions=param_dist, 
    n_iter=30,  # Fewer iterations for speed
    cv=3,  # 3-fold cross-validation (faster than 5-fold)
    scoring='accuracy', 
    n_jobs=-1,  # Use all CPU cores
    verbose=1, 
    random_state=42
)

# Fit on training data
random_search.fit(X_train, y_train)

# Print best parameters
print("Best Hyperparameters:", random_search.best_params_)

# Use the best model
best_rf = random_search.best_estimator_

Fitting 3 folds for each of 30 candidates, totalling 90 fits


c:\Users\Teresa\miniforge3\envs\ml\Lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


Best Hyperparameters: {'n_estimators': 300, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_depth': 20}


In [150]:
best_params = random_search.best_params_

# Train the final model with best parameters
rf_model = RandomForestClassifier(
    n_estimators=best_params['n_estimators'],
    max_depth=best_params['max_depth'],
    min_samples_split=best_params['min_samples_split'],
    min_samples_leaf=best_params['min_samples_leaf'],
    random_state=42
)

# Train the model on the full training data
rf_model.fit(X_train, y_train)

RandomForestClassifier(max_depth=20, n_estimators=300, random_state=42)

In [130]:
# Initialize the model
rf_model = RandomForestClassifier(n_estimators=500, random_state=42)

# Train the model
rf_model.fit(X_train, y_train)

RandomForestClassifier(n_estimators=500, random_state=42)

In [151]:
# Make predictions
y_pred = rf_model.predict(X_test)

### Evaluate

In [152]:
# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")
print(f"F1-Score: {f1_score(y_test, y_pred, average='weighted'):.2f}")

Accuracy: 0.88
F1-Score: 0.86


### Feature importance

In [153]:
# Feature importance
importances = rf_model.feature_importances_
feature_names = X.columns
importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances}).sort_values(by='Importance', ascending=False)

print("Feature Importances:")
print(importance_df)

Feature Importances:
          Feature  Importance
16       hotkey42    0.054124
32              s    0.052956
19       hotkey52    0.051980
13       hotkey32    0.051451
22       hotkey62    0.046774
0            Base    0.045986
10       hotkey22    0.045228
5        hotkey10    0.042366
7        hotkey12    0.041888
11       hotkey30    0.041634
14       hotkey40    0.039953
8        hotkey20    0.038934
33              0    0.036882
17       hotkey50    0.035036
20       hotkey60    0.032459
4        hotkey02    0.031227
2        hotkey00    0.029919
25       hotkey72    0.027410
23       hotkey70    0.026833
26       hotkey80    0.026178
1   SingleMineral    0.025756
28       hotkey82    0.024419
29       hotkey90    0.023343
6        hotkey11    0.022922
31       hotkey92    0.022556
18       hotkey51    0.012904
3        hotkey01    0.012642
15       hotkey41    0.012621
9        hotkey21    0.011539
24       hotkey71    0.007720
12       hotkey31    0.007226
30       hotkey91  

In [154]:
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(12,6))
plt.barh(importance_df['Feature'][:20], importance_df['Importance'][:20], color='royalblue')
plt.xlabel("Feature Importance Score")
plt.ylabel("Feature")
plt.title("Top 20 Most Important Features")
plt.gca().invert_yaxis()  # Flip to show most important features on top
#plt.show()

### Predict on test set

In [157]:
#load the calculated features on the test set
features_test = pd.read_csv('data/output/combined2.csv')
features_test.head(5)

,Base,SingleMineral,hotkey00,hotkey01,hotkey02,hotkey10,hotkey11,hotkey12,hotkey20,hotkey21,...,hotkey71,hotkey72,hotkey80,hotkey81,hotkey82,hotkey90,hotkey91,hotkey92,s,0
0,0.005810,0.000917,0.004281,0.0,0.259021,0.002141,0.009480,0.107645,0.003058,0.000000,...,0.0,0.013150,0.000000,0.0,0.000000,0.001835,0.0,0.008869,0.259327,0.019583
1,0.000000,0.000000,0.000902,0.0,0.014885,0.006315,0.000000,0.151556,0.013983,0.000000,...,0.0,0.000000,0.000000,0.0,0.000000,0.000451,0.0,0.021200,0.258006,0.032219
2,0.005719,0.000000,0.000286,0.0,0.000000,0.002860,0.000000,0.136975,0.015156,0.000000,...,0.0,0.009151,0.001430,0.0,0.014012,0.000000,0.0,0.000000,0.158993,0.046217
3,0.005640,0.000664,0.000000,0.0,0.000000,0.022561,0.029529,0.174187,0.004313,0.001991,...,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.526875,0.064677
4,0.039474,0.000000,0.005639,0.0,0.000000,0.007519,0.013158,0.139098,0.015038,0.000000,...,0.0,0.005639,0.007519,0.0,0.001880,0.000000,0.0,0.000000,0.443609,0.134545


In [158]:
# Compare column names
train_columns = set(X_train.columns)  # Columns from the training data
test_columns = set(features_test.columns)    # Columns from the test data

if train_columns != test_columns:
    print("Mismatch in columns!")
    print("Missing in test:", train_columns - test_columns)
    print("Extra in test:", test_columns - train_columns)

In [136]:
missing_columns = ['hotkey72.1', 'hotkey82.1', 'hotkey11.1', 'hotkey71.1']

# Add the missing columns and set their values to 0
for col in missing_columns:
    features_test[col] = 0

In [137]:
# Reorder the columns in 'df' to match the order in 'reference_df'
features_test = features_test[X_train.columns]

KeyError: "['0'] not in index"

In [159]:
#assert the column headers are in str format
features_test.columns = features_test.columns.astype(str)
#run prediction on test set
y_pred = rf_model.predict(features_test)

In [160]:
#save predictions to csv file
pd.DataFrame(y_pred).to_csv('data/predictions/predictions_kaggle4.csv', index=False)